In [ ]:
#Importing all the libraries

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib
import csv
import ortools
import math
import configparser
import datetime
import time
from geopy.geocoders import Nominatim
matplotlib.rc('xtick', labelsize=26) 
matplotlib.rc('ytick', labelsize=26) 
plt.rcParams['font.size'] = '26'
plt.rcParams['figure.figsize'] = (10,7.5)
fig, ax = plt.subplots()
fig.set_size_inches(10, 7.5)


configParser = configparser.RawConfigParser()   
configFilePath = 'config.cfg'
configParser.read(configFilePath)

In [ ]:
#File paths
configParser.read(configFilePath)
customer_dir=configParser.get('file-paths','customer_dir')
geolocation_dir=configParser.get('file-paths','geolocation_dir')
seller_dir = configParser.get('file-paths','seller_dir')
order_dir= configParser.get('file-paths','order_dir')
order_item_dir=configParser.get('file-paths','order_item_dir')


#Creating dataframes
customer_df=pd.read_csv(customer_dir)
location_df=pd.read_csv(geolocation_dir)
seller_df=pd.read_csv(seller_dir)
order_df=pd.read_csv(order_dir)
order_item_df=pd.read_csv(order_item_dir)

# location_df=location_df[location_df['geolocation_state']=='SP']
state_or_city=configParser.get('dataset-generation','state_or_city')
city_dataset= configParser.get('dataset-generation','city')
state_dataset= configParser.get('dataset-generation','state')
equal_or_not=int(configParser.get('dataset-generation','equal_to'))

if state_or_city=='state':
  if equal_or_not==1:
    location_df=location_df[location_df['geolocation_state']==state_dataset]
  else:
    location_df=location_df[location_df['geolocation_state']!=state_dataset]
else:
  if equal_or_not==1:
    location_df=location_df[location_df['geolocation_city']==city_dataset]
  else:
    location_df=location_df[location_df['geolocation_city']!=city_dataset]


location_df=location_df.drop_duplicates('geolocation_zip_code_prefix')


training_data_percent=int(configParser.get('dataset-generation','training_data_percent'))
club_num_dates=int(configParser.get('dataset-generation','club_num_dates'))

def club_date(dt,club_num_dates):
  return pd.datetime(dt.year,dt.month,int(dt.day/club_num_dates)*club_num_dates+1) 




#Dividing training and testing data
#order_df Datefram having data related to orders
order_df=order_df[order_df['order_status']=='delivered']
order_df=order_df[order_df['order_delivered_customer_date'].notna()]
order_df['order_delivered_customer_date']=pd.to_datetime(order_df['order_delivered_customer_date'])
order_df['order_delivered_customer_date']=order_df['order_delivered_customer_date'].dt.date
#Clubbing multiple dates into one, because the data for one date is too small
order_df['order_delivered_customer_date']=order_df['order_delivered_customer_date'].map(lambda x: club_date(x,club_num_dates))
order_df=order_df.sort_values("order_delivered_customer_date")
order_df=order_df.reset_index()



overall_customer_df=pd.merge(customer_df,order_df,on='customer_id')
overall_customer_loc_df=pd.merge(overall_customer_df,location_df,left_on='customer_zip_code_prefix' ,right_on='geolocation_zip_code_prefix')

#Dividing training data and testing data
start_training_date=order_df["order_delivered_customer_date"].iloc[int((10+training_data_percent)/100*len(order_df.index))]
end_training_date=order_df["order_delivered_customer_date"].iloc[int(0.1*len(order_df.index))]
training_order_df=order_df[order_df['order_delivered_customer_date']<=start_training_date]
training_order_df=training_order_df[order_df['order_delivered_customer_date']>=end_training_date]
num_training_days=training_order_df['order_delivered_customer_date'].nunique()


training_customer_df=pd.merge(customer_df,training_order_df,on='customer_id')
training_customer_df=training_customer_df.drop_duplicates('customer_unique_id')
temp_df=pd.merge(order_item_df,training_order_df,on='order_id')
training_seller_df=pd.merge(seller_df,temp_df,on='seller_id')
training_seller_df=training_seller_df.drop_duplicates('seller_zip_code_prefix')


training_customer_loc_df = pd.merge(training_customer_df, location_df,left_on = 'customer_zip_code_prefix' ,right_on='geolocation_zip_code_prefix')
training_seller_loc_df = pd.merge(training_seller_df, location_df,left_on = 'seller_zip_code_prefix' ,right_on='geolocation_zip_code_prefix')

training_customer_loc_df.rename(columns = {'customer_city':'city'}, inplace = True)
training_seller_loc_df.rename(columns = {'seller_city':'city'}, inplace = True)
   
   


training_customer_locations=training_customer_loc_df[['geolocation_lat','geolocation_lng','city']]
training_seller_locations=training_seller_loc_df[['geolocation_lat','geolocation_lng','city']]
training_combined_locations=pd.concat([training_customer_locations,training_seller_locations])







In [ ]:

#Plot customer, sellers and drivers
lis=['geolocation_lat','geolocation_lng']
X=training_customer_locations[lis].values
X1=np.transpose(X)
plt.scatter(X1[1],X1[0],color='Red',label='Customers')
X=training_seller_locations[lis].values
X1=np.transpose(X)
plt.scatter(X1[1],X1[0],color='Blue',label='Sellers')
plt.legend()
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Customers and Sellers locations')
plt.show()
X=training_combined_locations[lis].values
print("Number of customers = " + str(len(training_customer_locations.index)))
print("Number of sellers = " + str(len(training_seller_locations.index)))


#Generating FFCs locations using customer and seller data locations
configParser.read(configFilePath)
num_ffc_center=int(configParser.get('dataset-generation','num_ffc_center'))
kmeans=KMeans(n_clusters=num_ffc_center,random_state=0).fit(X)
centre=kmeans.cluster_centers_
label=pd.DataFrame(kmeans.labels_,columns=['label'])
label['freq']=label.groupby('label')['label'].transform('count')
label=label.drop_duplicates('label')
label=label.reset_index(drop=True)
label=label.sort_values('label')
X1=np.transpose(centre)
label['x']=X1[0]
label['y']=X1[1]


#Plot customer, sellers and drivers
lis=['geolocation_lat','geolocation_lng']
X=training_customer_locations[lis].values
X1=np.transpose(X)
plt.scatter(X1[1],X1[0],color='Red',label='Customers')
X=training_seller_locations[lis].values
X1=np.transpose(X)
plt.scatter(X1[1],X1[0],color='Blue',label='Sellers')
X1=np.transpose(centre)
plt.scatter(X1[1],X1[0],color='Black',label='FF centres')
plt.legend()
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Customers, Sellers and Warehouses locations')
plt.show()





In [ ]:
#Separating testing data
testing_order_df=order_df[order_df['order_delivered_customer_date']>end_training_date]
testing_customer_df=pd.merge(customer_df,testing_order_df,on='customer_id')
testing_customer_df=testing_customer_df.drop_duplicates('customer_unique_id')
testing_customer_loc_df = pd.merge(testing_customer_df, location_df,left_on = 'customer_zip_code_prefix' ,right_on='geolocation_zip_code_prefix')

testing_seller_df=pd.merge(seller_df,temp_df,on='seller_id')
testing_seller_df=testing_seller_df.drop_duplicates('seller_zip_code_prefix')
testing_seller_loc_df = pd.merge(testing_seller_df, location_df,left_on = 'seller_zip_code_prefix' ,right_on='geolocation_zip_code_prefix')



# removing outliers
lat_mean=training_combined_locations['geolocation_lat'].mean()
lat_std= training_combined_locations['geolocation_lat'].std()
testing_customer_loc_df=testing_customer_loc_df[(testing_customer_loc_df['geolocation_lat']-lat_mean)< 3*lat_std]
testing_customer_loc_df=testing_customer_loc_df[(testing_customer_loc_df['geolocation_lat']-lat_mean)> -3*lat_std]

lng_mean=training_combined_locations['geolocation_lng'].mean()
lng_std= training_combined_locations['geolocation_lng'].std()
testing_customer_loc_df=testing_customer_loc_df[(testing_customer_loc_df['geolocation_lng']-lng_mean)< 3*lng_std]
testing_customer_loc_df=testing_customer_loc_df[(testing_customer_loc_df['geolocation_lng']-lng_mean)> -3*lng_std]

testing_customer_loc_df=testing_customer_loc_df.reset_index(drop=True)


#Assigning ffc center to each customer as the nearest center according to prediction of kmeans distance
lis=['geolocation_lat','geolocation_lng']
prediction_array=np.array(kmeans.predict(testing_customer_loc_df[lis].values))
unique, counts = np.unique(prediction_array, return_counts=True)
testing_customer_loc_df['ffc_index']=prediction_array
testing_customer_loc_df=testing_customer_loc_df.sort_values('order_delivered_customer_date')



In [ ]:
# Finding maximum and minimum capacity of a warehouse for training data
prediction_array=np.array(kmeans.predict(training_customer_loc_df[lis].values))
unique, counts = np.unique(prediction_array, return_counts=True)
training_customer_loc_df['ffc_index']=prediction_array

# Finding the number of orders of particular ffc center for each data
training_customer_group_date = training_customer_loc_df.groupby(['ffc_index','order_delivered_customer_date'])['order_id'].count()

configParser.read(configFilePath)
quantile_capacity=float(configParser.get('dataset-generation','qauntile_capacity'))

training_customer_group_date=pd.DataFrame({'count':training_customer_group_date}).reset_index()
training_customer_group_date = training_customer_group_date.groupby(['ffc_index'])['count'].quantile(quantile_capacity)
training_customer_group_date = pd.DataFrame({'cap':training_customer_group_date}).reset_index()

training_customer_group_date['min_cap'] = 0.5 * training_customer_group_date['cap']
training_customer_group_date['max_cap'] = 1.5 * training_customer_group_date['cap']
training_customer_group_date=training_customer_group_date.sort_values('ffc_index')
max_cap=training_customer_group_date['max_cap'].values
min_cap=training_customer_group_date['min_cap'].values


In [ ]:


import random
def driver_generation(val):

  # Creating drivers through grid formation

  X=training_combined_locations[lis].values
  X=training_combined_locations[['geolocation_lat','geolocation_lng']].values
  X1=np.transpose(X)
  min_long=np.amin(X1[0])
  max_long=np.amax(X1[0])

  min_lat=np.amin(X1[1])
  max_lat=np.amax(X1[1])

  configParser.read(configFilePath)
  prop_constant_driver_customer=float(configParser.get('dataset-generation','prop_constant_driver_customer'))
  grid_length=int(configParser.get('dataset-generation','grid_length'))
  grid_width=int(configParser.get('dataset-generation','grid_width'))
  prop_constant_lower_range=float(configParser.get('dataset-generation','prop_constant_lower_range'))
  prop_constant_higher_range=float(configParser.get('dataset-generation','prop_constant_upper_range'))

  # random.seed(int(configParser.get('dataset-generation','Random_seed_value')))
  random.seed(val)

  grid_size=[grid_length,grid_width]
  drivers=[[],[]]
  sum_customer=0
  sum_driver=0



  

  for i in range(grid_size[0]):
    left_long=min_long+(max_long-min_long)/grid_size[0]*i
    right_long=min_long+(max_long-min_long)/grid_size[0]*(i+1)
    ind1=X1[0]>=left_long
    ind2=X1[0]<=right_long

    for j in range(grid_size[1]):
      up_lat=min_lat+(max_lat-min_lat)/grid_size[1]*(j+1)
      down_lat=min_lat+(max_lat-min_lat)/grid_size[1]*j
      ind3=X1[1]>=down_lat
      ind4=X1[1]<=up_lat
      ind=ind1 & ind2 & ind3 & ind4
     

      num_customers=np.count_nonzero(ind)/num_training_days
      num_drivers=num_customers/prop_constant_driver_customer
      random_constant=random.uniform(prop_constant_lower_range,prop_constant_higher_range)
      num_drivers=int(num_drivers*random_constant)

      if num_drivers==0:
        continue
      for k in range(num_drivers):
        drivers[0].append(random.uniform(left_long,right_long))
        drivers[1].append(random.uniform(down_lat,up_lat))
      

      sum_driver=sum_driver+num_drivers
      sum_customer=sum_customer+num_customers
  drivers=np.array(drivers)

  print("Number of Drivers:",len(drivers[0]))
  new_drivers=np.transpose(drivers)
  


  fig, ax = plt.subplots()
  tempdf=testing_customer_loc_df
  X=tempdf[lis].values
  X1=np.transpose(X)
  plt.scatter(X1[1],X1[0],color='Red',label='Customer')
  X=testing_seller_loc_df[lis].values
  X1=np.transpose(X)
  plt.scatter(X1[1],X1[0],color='Blue',label='Seller')
  X1=drivers
  plt.scatter(X1[1],X1[0],color='Cyan',label='Drivers')
  X1=np.transpose(centre)
  plt.scatter(X1[1],X1[0],color='Black',label='FF centres')
  plt.legend(loc='lower left',prop={'size':20})
  plt.xlabel('Longitude')
  plt.ylabel('Latitude')
  # plt.savefig("Plots/Coordinate Plots/"+state_dataset+".pdf")
  plt.show()
  plt.close()

  return new_drivers



driver_generation(12)






In [ ]:

#Using max flow for the Vanilla Distribution of Drivers to the warehouse
import networkx as nx
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a))


configParser.read(configFilePath)
prop_constant_driver_customer=float(configParser.get('dataset-generation','prop_constant_driver_customer'))

upper_cap=max_cap/prop_constant_driver_customer
lower_cap=min_cap/prop_constant_driver_customer
lower_cap=lower_cap.astype(int)
upper_cap=upper_cap.astype(int)
upper_cap=np.array(upper_cap)
lower_cap=np.array(lower_cap)
lower_cap[lower_cap<=0]=1







# Code to generate MCCA distribution using networkX library
def flow_with_max_capacity(driver_loc,centre):
  G = nx.DiGraph()
  num_variables=len(centre)+len(driver_loc)
  for i in range(len(driver_loc)):
    G.add_edge(1,i+2,capacity=1,weight=0)


  for i in range(len(centre)):
    G.add_edge(len(driver_loc)+1+(i+1),num_variables+2,capacity=int(upper_cap[i]),weight=0)



  for i in range(len(driver_loc)):
    for j in range(len(centre)):
      dis=(driver_loc[i][0]-centre[j][0])**2+(driver_loc[i][1]-centre[j][1])**2
      G.add_edge(i+2,len(driver_loc)+1+(j+1),capacity=1,weight=int(dis*1000))

  mincostFlow = nx.max_flow_min_cost(G, 1, num_variables+2)
  mincost = nx.cost_of_flow(G, mincostFlow)
  print(mincost)
  from networkx.algorithms.flow import maximum_flow
  maxFlow = maximum_flow(G, 1, num_variables+2)[1]
  print(nx.cost_of_flow(G, maxFlow))


  print(nx.maximum_flow_value(G,1,num_variables+2))
  driver_df=pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df['ffc_index']=-1
  for i in range(len(driver_loc)):
    for j in range(len(centre)):
      if abs(mincostFlow[i+2][len(driver_loc)+(j+2)]-1)<0.1:
        driver_df.at[i,'ffc_index']=j
  return driver_df


# Code to generate MCCA distribution using ortools library

from ortools.graph import pywrapgraph
def flow_with_maxmin_capacity(driver_loc,centre):
  # Instantiate a SimpleMinCostFlow solver.
  # Using min-max flow for the Vanilla Distribution of Drivers to the warehouse
  num_variables=len(centre)+len(driver_loc)
  min_cost_flow = pywrapgraph.SimpleMinCostFlow()
  for i in range(len(driver_loc)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(1,i+2,1,0)

  for i in range(len(driver_loc)):
    for j in range(len(centre)):
      dis=(driver_loc[i][0]-centre[j][0])**2+(driver_loc[i][1]-centre[j][1])**2
      min_cost_flow.AddArcWithCapacityAndUnitCost(i+2,len(driver_loc)+1+(j+1),1,int(dis*1000))


  for i in range(len(centre)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(len(driver_loc)+1+(i+1),num_variables+2,int(upper_cap[i])-int(lower_cap[i]),0)


  for i in range(0,len(centre)):
    min_cost_flow.SetNodeSupply(len(driver_loc)+1+(i+1),-int(lower_cap[i]))

  source_supply=int(len(driver_loc))
  sink_supply=int(len(driver_loc)-np.sum(lower_cap))

  min_cost_flow.SetNodeSupply(1,source_supply)
  min_cost_flow.SetNodeSupply(num_variables+2,-sink_supply)


  for i in range(0,len(driver_loc)):
    min_cost_flow.SetNodeSupply(i+2,0)


  if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
      print('Minimum cost:', min_cost_flow.OptimalCost())
      print('')
      print('  Arc    Flow / Capacity  Cost')
      for i in range(min_cost_flow.NumArcs()):
        if min_cost_flow.Flow(i)!=0:
          cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
          print('%1s -> %1s   %3s  / %3s       %3s' % (
              min_cost_flow.Tail(i),
              min_cost_flow.Head(i),
              min_cost_flow.Flow(i),
              min_cost_flow.Capacity(i),
              cost))
  else:
    print('There was an issue with the min cost flow input.')

  driver_df1=pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df1['ffc_index']=-1
  for i in range(min_cost_flow.NumArcs()):
    if min_cost_flow.Flow(i)!=0 and min_cost_flow.Tail(i)!=1 and min_cost_flow.Head(i)!=(num_variables+2):
      tail=min_cost_flow.Tail(i)
      head=min_cost_flow.Head(i)
      driver_df1.at[tail-2,'ffc_index']=head-len(driver_loc)-2

  return driver_df1








In [ ]:
#Fair Clustering - LPP contstraints and Cplex
from cplex import Cplex
from lp_tools import *

configParser.read(configFilePath)
fair_distance=int(configParser.get('fairness-constraint','fair_distance'))
alpha_fair =float(configParser.get('fairness-constraint','alpha_fair'))
  


def fair_clustering(dataset, centres):

  # Step 1: 	Create an instance of Cplex 
  problem = Cplex()

  # Step 2: 	Declare that this is a minimization problem
  problem.objective.set_sense(problem.objective.sense.minimize)

  # Step 3.   Declare and  add variables to the model. The function
  #           prepare_to_add_variables (dataset, centres) prepares all the
  #           required information for this stage.
  #
  #    objective: a list of coefficients (float) in the linear objective function
  #    lower bound: a list of floats containing the lower bounds for each variable
  #    upper bound: a list of floats containing the upper bounds for each variable
  #    variable_names: a list of strings that contains the name of the variables

  objective, lower_bound, upper_bound, variable_names, P,C = prepare_to_add_variables(dataset, centres)
  problem.variables.add(
      obj = objective,
      lb = lower_bound,
      ub = upper_bound,
      names = variable_names
     
    )

  # Step 4.   Declare and add constraints to the model.
  #           There are few ways of adding constraints: row wise, col wise and non-zero entry wise.
  #           Assume the constraint matrix is A. We add the constraints non-zero entry wise.
  #           The function prepare_to_add_constraints(dataset, centres)
  #           prepares the required data for this step.
  #
  #  coefficients: Three tuple containing the row number, column number and the value of the constraint matrix
  #  senses: a list of strings that identifies whether the corresponding constraint is
  #          an equality or inequality. "E" : equals to (=), "L" : less than (<=), "G" : greater than equals (>=)
  #  rhs: a list of floats corresponding to the rhs of the constraints.
  #  constraint_names: a list of string corresponding to the name of the constraint
 
  rhs, senses, row_names, coefficients = prepare_to_add_constraints(dataset, centres, upper_cap,lower_cap, P,C, alpha_fair,fair_distance)
  print("NUM_constraints",len(senses))
  problem.linear_constraints.add(
      rhs = rhs,
      senses = senses,
      names = row_names
    )
  problem.linear_constraints.set_coefficients(coefficients)

  # Step 5.	Solve the problem
  problem.solve()

  result = {
    "status": problem.solution.get_status(),
    "success": problem.solution.get_status_string(),
    "objective": problem.solution.get_objective_value(),
    "assignment": problem.solution.get_values(),
  }

  return result









In [ ]:

#Income Distribution Graph for Warehouses

configParser.read(configFilePath)

# Generates box plot for 2d income list
def box_plot(income_2d,city_lis):
  fig, ax = plt.subplots()
  plt.boxplot(income_2d)
  ax.set_xticklabels(city_lis,rotation='vertical')
  plt.ylabel('Number of Deliveries')
  plt.ylim((0,1000))
  ax.set_ylim([0, 700])
  plt.show()
  

# Generates scatter income plot, labeling drivers according to the range in which their income lies
def scatter_income(driver_income_df,fair_type):
  fig, ax = plt.subplots()
  fig.set_size_inches(13, 9)
  configParser.read(configFilePath)

  diff_num=int(configParser.get('results-parameter','diff_num'))
  driver_income_df=driver_income_df.sort_values('total_income')
  driver_income_df=driver_income_df.reset_index()

  df_len=len(driver_income_df.index)
  low_list=[0,45,90,135]
  high_list=[45,90,135,100000]
  colors=['red','orange','green','blue']

  for i in range(diff_num):
    low_val=low_list[i]
    high_val=high_list[i]
    print(low_val,high_val)
    ind1=(driver_income_df['total_income']<=high_val)
    ind2=(driver_income_df['total_income']>low_val)
    tempo_df=driver_income_df[ind1 & ind2]

    label_string=str(low_val)
    if i==3:
      label_string=label_string+"+"
    else:
      label_string=label_string+" - "+str(high_val)
    
    plt.scatter(tempo_df['geolocation_lng'],tempo_df['geolocation_lat'],color=colors[i],label=label_string)
  plt.legend(loc='upper left',prop={'size':20})
  plt.xlabel('Longitude',fontsize='26')
  plt.ylabel('Latitude',fontsize='26')

  if fair_type=="fair_algo":
    plt.savefig("Plots/Scatter Plots/"+state_dataset+"_result.pdf")
  else:
    plt.savefig("Plots/Scatter Plots/"+state_dataset+"_motivation.pdf")
  print(driver_income_df['total_income'].values)
  

  plt.show()
    


def lorenz(arrs,name_dis,colors):
    # this divides the prefix sum by the total sum
    # this ensures all the values are between 0 and 1.0
    import matplotlib.pyplot as plt1
    fig, ax = plt1.subplots()
    fig.set_size_inches(10, 7.5)
    plt1.rcParams['font.size'] = '26'
    plt1.rcParams['figure.figsize'] = (10,7.5)
    

    for i in range(len(arrs)):
      arr=arrs[i]
      arr=np.array(arr)
      arr=np.sort(arr)
      plt.rcParams.update({'font.size': 14})
      scaled_prefix_sum = arr.cumsum() / arr.sum()*100
      # this prepends the 0 value (because 0% of all people have 0% of all wealth)
      lorenz_cruve= np.insert(scaled_prefix_sum, 0, 0)
      # we need the X values to be between 0.0 to 1.0

      plt1.plot(np.linspace(0.0, 100.0, lorenz_cruve.size), lorenz_cruve,label=name_dis[i],color=colors[i])
    
    plt1.plot([0,100],[0,100],color='Black',linestyle='--')
    plt1.legend(loc='upper left',prop={'size':20})
   
    plt1.xlabel('% of all drivers',fontsize=26)
    plt1.ylabel('% of total deliveries',fontsize=26)
    if len(name_dis)==2:
      plt1.savefig("Plots/Lorenz Curves/"+state_dataset+"_motivation.pdf")
    else:
      plt1.savefig("Plots/Lorenz Curves/"+state_dataset+"_result.pdf")



    
# Caluclate average distance each driver has to travel each day from going to home to the location of FF Center
def average_distance(driver_dist,label):   
    distance_warehouse_driver_df = pd.merge(driver_dist, label,left_on = 'ffc_index' ,right_on='label')
    dist1=0.0
    for i in range(len(distance_warehouse_driver_df)):
        dist1 = dist1 +math.sqrt((distance_warehouse_driver_df.loc[i].at["geolocation_lat"] - distance_warehouse_driver_df.loc[i].at["x"])**2 + (distance_warehouse_driver_df.loc[i].at["geolocation_lng"] - distance_warehouse_driver_df.loc[i].at["y"])**2)
    return round(dist1/len(distance_warehouse_driver_df),2)


    
# Returns gini index on income list
def gini_index(incomes):
  driver_income=np.array(incomes)
  driver_income=np.sort(driver_income)
  integer_array=np.array([(i+1) for i in range(len(driver_income))])
  driver_income=driver_income*750
  gini_index=2*(np.dot(driver_income,integer_array))/(len(driver_income)*(np.sum(driver_income))) - 1 -1/(len(driver_income))
  return round(gini_index,3)



  

In [ ]:
def inequality(df,date_index):
  #Finding the number of drivers assigned to every ff center
  df['driver_index']=np.arange(len(df.index))  
  driver_assign = df.groupby('ffc_index')['geolocation_lat'].count()
  driver_assign=pd.DataFrame({'num_drivers':driver_assign}).reset_index()

  # Adding 0 driver count to center to whom no driver was assigned
  for i in range(len(centre)):
    if i not in driver_assign['ffc_index'].values:
      driver_assign.loc[len(driver_assign.index)]=[i,0]


  driver_assign=driver_assign.sort_values('ffc_index')
  driver_assign['min_cap']=lower_cap
  driver_assign['max_cap']=upper_cap

  # Finding income of each driver as total number of customers divided by total number of drivers in that center
  # For a particular data: dates[date_index] 

  #Finding the income of each warehouse
  testing_customer_group_date = testing_customer_loc_df.groupby(['ffc_index','order_delivered_customer_date'])['order_id'].count()
  testing_customer_group_date = pd.DataFrame({'num_delivery_per_date':testing_customer_group_date}).reset_index()

  dates=testing_customer_group_date['order_delivered_customer_date'].unique()
  testing_customer_group_date=testing_customer_group_date[testing_customer_group_date['order_delivered_customer_date']==dates[date_index]]

  warehouse_income_df = pd.merge(testing_customer_group_date, driver_assign,on='ffc_index',how='right')
  warehouse_income_df['income_warehouse_driver'] = warehouse_income_df['num_delivery_per_date'] / warehouse_income_df['num_drivers']
  warehouse_income_df.loc[~np.isfinite(warehouse_income_df['income_warehouse_driver']), 'income_warehouse_driver'] = 0
  warehouse_income_df=warehouse_income_df.round(2)
  warehouse_income_df.loc[warehouse_income_df['num_drivers']==0,'income_warehouse_driver']=0
  warehouse_income_df.reset_index(inplace=True)

  warehouse_incomes=warehouse_income_df['income_warehouse_driver'].values

  df['date']=dates[date_index]
  df['income']=-1.0
  

  # Assigning the income of ff center to which the driver was assigned
  for i in range(len(df.index)):
    df.at[i,'income']=warehouse_income_df.at[df.iloc[i]['ffc_index'],'income_warehouse_driver']
  return df,warehouse_incomes
 


In [ ]:
#Fair Assignment of drivers to the FFCs/ warehouses
import dependent_routing as dp
configParser.read(configFilePath)

# Assigning the driver using the probaility distribution using dependent routing
def fair_assignment(prob_dis,driver_loc):
  
  final_ans1=copy.deepcopy(prob_dis)
  print(final_ans1.shape)
  routing=dp.DependentRounding(final_ans1)
  routing._buildGraph(final_ans1)
  final_assignment=routing.round()
  final_assignment=np.around(final_assignment,2)

  driver_df2=pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df2['ffc_index']=-1
  for i in range(num_samples):
    for j in range(num_centres):
      if abs(final_assignment[i][j]-1)<0.01:
        driver_df2.at[i,'ffc_index']=j
  return driver_df2,final_assignment


# Randomly assigning the drivers to ff center keeping the upper_cap constraint
def random_dist(driver_loc):
  driver_df4=pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df4['ffc_index']=-1
  temp_upper_cap=list(upper_cap)
  for i in range(num_samples):
    ffc=random.randint(1,num_centres)-1
    while(temp_upper_cap[ffc]<=0 ):
      ffc=random.randint(1,num_centres)-1
    driver_df4.at[i,'ffc_index']=ffc
    temp_upper_cap[ffc]=temp_upper_cap[ffc]-1
  return driver_df4

# Assign the drivers in round robin manner to ff center keeping the upper_cap constraint
def round_robin_dist(d,driver_loc):
  driver_df5=pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df5['ffc_index']=-1
  temp_upper_cap=list(upper_cap)
  for i in range(num_samples):
    ffc=(d+i)%num_centres
    while(temp_upper_cap[ffc]<=0):
      ffc=(ffc+1)%num_centres
    driver_df5.at[i,'ffc_index']=ffc
    temp_upper_cap[ffc]=temp_upper_cap[ffc]-1
  return driver_df5

# Assign the drivers with LIPA manner to ff center keeping the upper_cap constraint
def low_income_dist(prev_income_drivers,prev_incomes_warehouse,driver_loc,lpp_prob_dis):
  if len(prev_income_drivers)==0:
    return fair_assignment(lpp_prob_dis,driver_loc)[0]
  driver_df6=pd.DataFrame(driver_loc,columns=["geolocation_lat","geolocation_lng"])
  driver_df6['ffc_index']=-1

  temp_upper_cap=list(upper_cap)
  driver_index_inc=np.argsort(np.array(prev_income_drivers))
  warehouse_index_inc=np.argsort(np.array(prev_incomes_warehouse))
  j=num_centres-1
  for i in driver_index_inc:
    ffc=warehouse_index_inc[j]
    while(temp_upper_cap[ffc]<=0):
      j=j-1
      ffc=warehouse_index_inc[j]
    driver_df6.at[i,'ffc_index']=ffc
    temp_upper_cap[ffc]=temp_upper_cap[ffc]-1
  return driver_df6

# Returns the total cost of the drivers from their location to ff center
def total_cost(df):
  sum=0
  for index,row in df.iterrows():
    sum=sum+ np.sqrt( (row['geolocation_lat']-centre[int(row['ffc_index'])][0])**2 + (row['geolocation_lng']-centre[int(row['ffc_index'])][1])**2 )
  return round(sum,2)



In [ ]:
# Finding the income of all drivers for 30 days, with given distribution type
def inequality2(algo_type="random",num_days=30,lpp_prob_dis=[[]]):
  fair_assignment_totaldays=None
  overall_cost=0
  prev_income_drivers=[]
  prev_incomes_warehouse=[]
  df=None
  if algo_type=="vanilla_max":
    df_vanilla_max=flow_with_max_capacity(new_drivers,centre)
  if algo_type=="vanilla_maxmin":
    df_vanilla_maxmin=flow_with_maxmin_capacity(new_drivers,centre)

  # Calling appropriate function according to the given algo_type paramtric value
  for i in range(num_days):
    if algo_type=="random": df=random_dist(new_drivers)
    if algo_type=="round_robin": df=round_robin_dist(i,new_drivers)
    if algo_type=="fair_algo": df=fair_assignment(lpp_prob_dis,new_drivers)[0]
    if algo_type=="vanilla_max": df=df_vanilla_max   
    if algo_type=="vanilla_maxmin": df=df_vanilla_maxmin  
    if algo_type=="low_income_dist":df=low_income_dist(prev_income_drivers,prev_incomes_warehouse,new_drivers,lpp_prob_dis)
    

    df_result,prev_incomes_warehouse=inequality(df,i)
    overall_cost=overall_cost+total_cost(df)

    if fair_assignment_totaldays is None:
      fair_assignment_totaldays=df_result
    else:
      fair_assignment_totaldays=pd.concat([fair_assignment_totaldays,df_result])
    prev_income_drivers=fair_assignment_totaldays.groupby('driver_index')['income'].sum().values


  # Finding total income of drivers over all days  
  lis1=fair_assignment_totaldays.groupby('driver_index')['income'].sum().values
  fair_assignment_totaldays['total_income'] = fair_assignment_totaldays['income'].groupby(fair_assignment_totaldays['driver_index']).transform('sum')
  
  driver_income_df=fair_assignment_totaldays.groupby('driver_index').first().reset_index()  
  avg_distance=average_distance(fair_assignment_totaldays,label)
  return gini_index(lis1),avg_distance,fair_assignment_totaldays,np.sum(np.array(lis1)),lis1


  

In [ ]:

# Find the income gap per pair of those drivers which are separated by less than fair_distance
def helper_func(table,driver_loc):
    income_gap = 0.0
    assignment_total = table.groupby(['driver_index','geolocation_lat','geolocation_lng'])['income'].sum()
    assignment_total=pd.DataFrame({'income_sum':assignment_total}).reset_index()
    income_array = []
    for i in range(len(driver_loc)):
        income_array.append(assignment_total.loc[i].at["income_sum"])
    income_array = np.array(income_array)

    transposed = np.expand_dims(driver_loc, axis = 1)
    distance_drivers = np.power(driver_loc - transposed, 2)
    distance_drivers = np.power(np.abs(distance_drivers).sum(axis = 2), 0.5) 
    distance_drivers=distance_drivers*110

    transposed = np.expand_dims(income_array, axis = 1)
    income_diff = np.abs(income_array - transposed)

    configParser.read(configFilePath)
    fair_distance=int(configParser.get('fairness-constraint','fair_distance'))
    
    num_pair_fair_drivers=1e-7
    for i in range(len(driver_loc)-1):
        for j in range(i+1,len(driver_loc)):
            if(distance_drivers[i][j] <=fair_distance):
                income_gap = income_gap + float(income_diff[i][j])/distance_drivers[i][j]
                num_pair_fair_drivers=num_pair_fair_drivers+1
    
    return round(income_gap/num_pair_fair_drivers,2),income_diff,distance_drivers,assignment_total


In [ ]:
# Returns spatial inequality.
def spatial_inequality(income_diff, distance_drivers,table,driver_loc):
    sum_i = 0.0
    total_sum = 0.0
    configParser.read(configFilePath)
    fair_distance=int(configParser.get('fairness-constraint','fair_distance'))
    for i in range(len(driver_loc)):
        sum_i = sum_i + table.loc[i].at["income_sum"]
        sum_i_j = 0.0
        num_i_j = 1e-7
        for j in range(i+1,len(driver_loc)):
            dist = distance_drivers[i][j]
            if (dist < fair_distance):
                sum_i_j = sum_i_j + income_diff[i][j]
                num_i_j = num_i_j + 1
        total_sum = total_sum + float(sum_i_j)/num_i_j
    sp_index = float(total_sum)/sum_i
    return round(sp_index,4)

def fraction(base,fair):
    fair_income_array=fair["income_sum"].values
    base_income_array=base["income_sum"].values
    return np.around((np.divide(fair_income_array,(base_income_array+1e-5))-1)*100,2)
    


In [ ]:
Gini_Index_List=[]
Avg_Distance_List=[]
Income_Gap_List=[]
spatial_inequality_list=[]
Name_list=[]
Income_sum_lis=[]
num_drivers=[]
income_lis_lis=[]
fraction_list=[]
import copy



# Do a sanity check on the distribution we get from FairAssign algorithm
# Flag if the value is less than -1e-5, otherwise for other negative values convert them to 0
def sanityCheck(weights):
    for i in range(len(weights)):
        last_pos_index=-1
        neg_value=0
        for j in range(len(weights[0])):
            assert weights[i][j]>-1e-5
            if weights[i][j]<0:
                neg_value+=weights[i][j]
                weights[i][j]=0
            elif weights[i][j]>0:
                last_pos_index=j
        weights[i][last_pos_index]+=neg_value
    return weights


configParser.read(configFilePath)
number_of_runs=int(configParser.get('dataset-generation','number_of_runs'))
random_seed=int(configParser.get('dataset-generation','random_seed'))

dis_type=["random","low_income_dist","round_robin","vanilla_max","vanilla_maxmin","fair_algo"]
Name_dis_types=["Random","LIPA","RoundRobin","MCCA","MCCA-L","FairAssign"]
colors=['Indigo','Blue','Cyan','Green','Orange','Red']

indexs_allowed=[0,1,2,3,4,5]

# Generates result for #number_of_runs times, and store the value in differnt list
for i in range(number_of_runs):
    new_drivers=driver_generation(i+random_seed)
    if len(new_drivers)<sum(lower_cap):
        print("Lower number of drivers generated")
        print(len(new_drivers), sum(lower_cap))
        continue

    result=fair_clustering(new_drivers,centre)
    num_samples = len(new_drivers)
    num_centres = len(centre)

    #Probability Distribution output from cplex
    lpp_prob_dis=np.reshape(result['assignment'][:num_samples*num_centres],(-1,num_centres))
    lpp_prob_dis=sanityCheck(copy.deepcopy(lpp_prob_dis))

    print("Everything solved, graphs start")
    income_lis_lis=[]
    

    for j in indexs_allowed:
        tup=inequality2(algo_type=dis_type[j],lpp_prob_dis=lpp_prob_dis)
        tup2 = helper_func(tup[2],new_drivers)
        spat=spatial_inequality(tup2[1],tup2[2],tup2[3],new_drivers)

        Gini_Index_List.append(tup[0])
        Avg_Distance_List.append(tup[1])
        Income_Gap_List.append(tup2[0])
        spatial_inequality_list.append(spat)
        Name_list.append(Name_dis_types[j])
        Income_sum_lis.append(tup[3])
        num_drivers.append(num_samples)
        income_lis_lis.append(tup[4])

        if j==3:
            base=tup2[3]
        if j==5:
            fair=tup2[3]

    fraction_worse_off = fraction(base,fair)
    fraction_list.append(fraction_worse_off)


    All_Income=pd.DataFrame({'Dist Type':Name_list,'Gini': Gini_Index_List, 'Avg Dist':Avg_Distance_List,'Income_Gap':Income_Gap_List,'spatial_index':spatial_inequality_list})
    print(All_Income.groupby('Dist Type').mean())
    print()
    print(All_Income.groupby('Dist Type').max())
    print()
    print(All_Income.groupby('Dist Type').min())
    print()












In [ ]:
# Find the fractional number of people worse off or get better when applied FairAssign algorithm keeping base as MCCA
from statistics import mean

greater_than_threshold=[40,35,30,25,20,15,10,5,0]
lesser_than_threshold=[0,-5,-10,-15,-20,-25,-30,-35,-40]
greater_than_matrix=[[0 for j in range(len(fraction_list))] for i in range(len(greater_than_threshold))]
lesser_than_matrix=[[0 for j in range(len(fraction_list))] for i in range(len(lesser_than_threshold))]

for i in range(len(fraction_list)):
    for j in range(len(greater_than_threshold)):
        val=(fraction_list[i]>=greater_than_threshold[j]).sum()/len(fraction_list[i])
        greater_than_matrix[j][i]=val
for i in range(len(fraction_list)):
    for j in range(len(lesser_than_threshold)):
        val=(fraction_list[i]<lesser_than_threshold[j]).sum()/len(fraction_list[i])
        lesser_than_matrix[j][i]=val

avg_greater_than=[]
avg_lesser_than=[]
for i in range(len(greater_than_matrix)):
    avg_greater_than.append(sum(greater_than_matrix[i])/len(greater_than_matrix[i]))
for i in range(len(lesser_than_matrix)):
    avg_lesser_than.append(sum(lesser_than_matrix[i])/len(lesser_than_matrix[i]))


print(greater_than_threshold+lesser_than_threshold)

Change_df=pd.DataFrame({'Change': greater_than_threshold+lesser_than_threshold,'Val':avg_greater_than+avg_lesser_than})
print(Change_df)


In [ ]:

# Print all the result
All_Income=pd.DataFrame({'Dist Type':Name_list,'Gini': Gini_Index_List, 'Avg Dist':Avg_Distance_List,'Income_Gap':Income_Gap_List,'spatial_index':spatial_inequality_list,'Income_sum':Income_sum_lis})

All_Income=All_Income.replace("Low Income Distribution","Robinhood")
configParser.read(configFilePath)
print("State: ",configParser.get('dataset-generation','state'))
print("Equal_to_or_not: ",configParser.get('dataset-generation','equal_to'))
print("num_ffc_center: ",configParser.get('dataset-generation','num_ffc_center'))
print("Grid Size: ",configParser.get('dataset-generation','grid_length'),configParser.get('dataset-generation','grid_width'))
print("Fair Distance: ",configParser.get('fairness-constraint','fair_distance'))
print("Alpha_fair: ",configParser.get('fairness-constraint','alpha_fair'))
print("number_of_runs: ",configParser.get('dataset-generation','number_of_runs'))
print()
print("Avg Num of Drivers: ",np.mean(np.array(num_drivers)))


print()
print(All_Income.groupby('Dist Type').mean().round(4))
print()
print(All_Income.groupby('Dist Type').max())
print()
print(All_Income.groupby('Dist Type').min())
print()